# Ejemplo de uso

In [1]:
import mib_v2_3_1 as mb

## Modelo de un problema

$P(ABC) = P(A)P(B|A)P(C|AB)$.

### Probabilidades

<table width="90%">
    <tr>
        <td valign="top">
            <table width="50%">
              <tr>
                <th> $A$</th>
                <th><p style="text-align:center;"> $P(A)$</p></th>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>
        </td>
        <td valign="top">
            <table width="60%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($B$|$A$)</p></td>
              </tr>
              <tr>
                <td>$A$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>        
        </td>
        <td valign="top">
            <table width="90%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($C$|$AB$)</p></td>
              </tr>
              <tr>
                <td>$AB$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$00$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$01$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$10$</p></td>
                <td><p style="text-align:center;">$0.4$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$11$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
            </table>        
        </td>
    </tr>
</table>

## Creación de datos

### Declaración de diccionarios

In [2]:
# Tablas
# {(variables independientes): {variables: p}}
# {(A,): P(A)}
dA = {
    (0,): 0.3,
    (1,): 0.7
}

# {(A,):{(B,): P(B|A)}}
dB_A = {
    (0,):{(0,): 0.2, (1,): 0.8},
    (1,):{(0,): 0.3, (1,): 0.7}
}

# {(A,B):{(C,): P(C|A,B)}}
dC_AB = {
    (0,0): {(0,): 0.1, (1,): 0.8, (2,): 0.1},
    (0,1): {(0,): 0.3, (1,): 0.5, (2,): 0.2},
    (1,0): {(0,): 0.4, (1,): 0.5, (2,): 0.1},
    (1,1): {(0,): 0.2, (1,): 0.7, (2,): 0.2}
    }


### Declaración de variables

In [3]:
A = mb.Var(name='A', values=set([0,1]))
B = mb.Var(name='B', values=set([0,1]))
C = mb.Var(name='C', values=set([0,1,2]))

### Declaración de distribuciones 

In [4]:
# P(A)
PA = mb.Distrib(table=dA, columns=(A.getName(),))
# P(B|A)
PB_A = mb.CondDistrib(table=dB_A, columns_vars=(B.getName(),), columns_indep=(A.getName(),))
# P(C|AB)
PC_AB = mb.CondDistrib(dC_AB, (C.getName(),), (A.getName(), B.getName()))

## Creación de la pregunta

In [5]:
# Probabilidad conjunta (Especidicación)
PABC = mb.Specification(set([A,B,C]), set([PA, PB_A, PC_AB]))
# Generador de consultas (Pregunta)
QABC = mb.Question(PABC)

## Creación de consultas

### Distribuciones

#### De una variable

In [6]:
PB = QABC.DistributionQuery(set([B]))
PB._table

{(0,): 0.27, (1,): 0.7789999999999999}

#### Conjuntas

In [7]:
PAB = QABC.DistributionQuery(set([A,B]))
PAB._table

{(0, 0): 0.060000000000000005,
 (0, 1): 0.21,
 (1, 0): 0.24000000000000002,
 (1, 1): 0.5389999999999999}

#### Condicional

In [8]:
PA_B = QABC.DistributionQuery(set([A]), set([B]))
print(PA_B._columns_indep, PA_B._columns_vars)
PA_B._table

('B',) ('A',)


{(0,): {(0,): 0.22222222222222224, (1,): 0.7777777777777777},
 (1,): {(0,): 0.30808729139922986, (1,): 0.6919127086007701}}

#### Consultas usando paralelismo

In [9]:
PA_B_mp = QABC.DistributionQuery_mp(set([A]), set([B]))
PA_B_mp._table

{(0,): {(0,): 0.22222222222222224, (1,): 0.7777777777777777},
 (1,): {(0,): 0.30808729139922986, (1,): 0.6919127086007701}}

In [10]:
PA_B_mp = QABC.DistributionQuery_mp(set([A]), set([B]), lote_n=1000, process_n=4)
PA_B_mp._table

{(0,): {(0,): 0.22222222222222224, (1,): 0.7777777777777777},
 (1,): {(0,): 0.30808729139922986, (1,): 0.6919127086007701}}

### Consulta de probabilidades

In [11]:
# P(A=0|C=0)
p = QABC.Query((A,), (C,), (0,), (0,))
p

0.30000000000000004

In [12]:
# P(A=0,B=0)
p = QABC.Query(vars=(A,B), vars_values=(0,0))
p

0.060000000000000005

### Consultas de valores más probables

In [13]:
#P(A|B=0)
p = QABC.Query(vars=(A,), indep=(B,), indep_values=(0,))
p

(('A',), (1,), 0.7777777777777777)

In [14]:
#P(A=0|B)
p = QABC.Query(vars=(A,), indep=(B,), vars_values=(0,))
p

(('B',), (1,), 0.30808729139922986)

In [15]:
#P(AB)
p = QABC.Query(vars=(A,B))
p

(('B', 'A'), (1, 1), 0.5389999999999999)